# Метод Prophet. Математическая модель.

Источник: [Лекция 1. Прогнозирование. Метод Prophet. Математическая модель.][1]

[1]: https://www.youtube.com/watch?v=Vj8bac2RNA8

## Заметки к лекции

Prophet  - модель опубликована в 2017 Taylor, Letham, Forecasting at scale

$Y_t = T_t + S_t + X_t + \epsilon_t$

аддитивная регрессионная модель с четырьмя основными компонентами:

- **тренд** == кусочно-линейной регрессии или кусочно логистической кривой; (почти всегда используется кусочно-линейная)
- **годовая сезонность** == ряд Фурье (автоматически или ручками);
- **недельная сезонность** == индикаторные переменные недели
- **интервенции** официальные праздничные дни и выходные дни - Новый год, Рождество и т.п., а также другие дни во время которых свойства временного ряда могу существенно измениться - спортивные или культурные события, природные явления и т.п.; как и в случае с днями недели, такие дни представлены в модели в виде индикаторных переменных


О рядах Фурье с картинками 
https://www.jezzamon.com/fourier

Оценивание параметров подгоняемой модели выполняется с использованием принципов байесовской статистики (либо методом нахождения апостериорного максимума (MAP), либо путем полного байесовского вывода). Для этого применяется платформа вероятностного программирования Stan.

Prophet представляет собой удобный интерфейс для работы с этой платформой
- из среды R
- посредством библиотеки fbprophet для Python.

В таблицах всегда:
- столбцу с датами присвоено имя ds,
- столбцу со значениями временного ряда - имя y.


## Заметки и код к [лекции][2] на тему: Метод Prophet. Библиотека prophet. Пример построения. Параметры процедуры Prophet.

[2]: https://www.youtube.com/watch?v=Ppi4NBS-hDk

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
df = pd.read_csv('../data/kaggle-supermarket-sales/supermarket_sales.csv')

In [6]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [4]:
df.columns

Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating'],
      dtype='object')

In [15]:
df[df['Branch']=='A']['Date'].unique()

array(['1/5/2019', '3/3/2019', '1/27/2019', '2/8/2019', '2/25/2019',
       '1/10/2019', '2/12/2019', '2/7/2019', '3/29/2019', '3/11/2019',
       '1/1/2019', '1/21/2019', '2/17/2019', '3/2/2019', '3/22/2019',
       '3/10/2019', '3/15/2019', '1/15/2019', '2/10/2019', '3/27/2019',
       '1/20/2019', '3/6/2019', '3/13/2019', '1/24/2019', '2/2/2019',
       '1/22/2019', '1/12/2019', '1/2/2019', '3/8/2019', '3/23/2019',
       '3/28/2019', '1/19/2019', '2/14/2019', '1/18/2019', '2/18/2019',
       '3/16/2019', '1/29/2019', '2/20/2019', '2/1/2019', '1/3/2019',
       '3/21/2019', '3/4/2019', '3/19/2019', '1/25/2019', '2/21/2019',
       '3/20/2019', '3/9/2019', '1/11/2019', '1/23/2019', '1/7/2019',
       '1/6/2019', '1/31/2019', '2/5/2019', '2/13/2019', '2/9/2019',
       '3/12/2019', '1/4/2019', '2/15/2019', '2/3/2019', '3/18/2019',
       '3/26/2019', '3/30/2019', '3/5/2019', '3/7/2019', '1/17/2019',
       '3/25/2019', '3/14/2019', '2/27/2019', '2/4/2019', '1/26/2019',
       '3/24/20